# Optymalizacja parametrów układu elektronicznego
Napięcia na mostku:

* U1 = 6V
* U2 = 10V
* U3 = 4V
* U4 = 7V
* U5 = 3V
    
Prądy mostka - zmienne w zakresach +/-Di = 1mA od Ii:
    
* I1 = 4mA
* I2 = 2mA
* I3 = 2mA
* I4 = 2mA
* I5 = 4mA

## Minimalizacja mocy wydzielanej na rezysotrach

Plik: Rezystory1.png

Podane są wartości napięć na poszczególnych elementach. Należy określić jakie powinny być wartości rezystancji, aby moc wydzielana w układzie była minimalna. 

![](Rezystory1.png)

In [7]:
from pulp import *

# problem statement
minimalizacjaStrat = LpProblem("Minimalizacja strat mocy na rezystorach",LpMinimize)

# zmienne decyzyjne
I1 = LpVariable("Prad 1",0,None,LpContinuous)
I2 = LpVariable("Prad 2",0,None,LpContinuous)
I3 = LpVariable("Prad 3",0,None,LpContinuous)
I4 = LpVariable("Prad 4",0,None,LpContinuous)
I5 = LpVariable("Prad 5",0,None,LpContinuous)

# wyrażenie do optymalizacji
minimalizacjaStrat += 6*I1 + 10*I2 + 4*I3 + 7*I4 + 3*I5, "MocTracona"

# ograniczenia (kazdy prad musi miec wartosc z przedzialu [Ii - Di, Ii + Di])
minimalizacjaStrat += 3 <= I1, "Ograniczenie_I1_min"   
minimalizacjaStrat += I1 <= 5, "Ograniczenie_I1_max"

minimalizacjaStrat += 1 <= I2, "Ograniczenie_I2_min"
minimalizacjaStrat += I2 <= 3, "Ograniczenie_I2_max"

minimalizacjaStrat += 1 <= I3, "Ograniczenie_I3_min"
minimalizacjaStrat += I3 <= 3, "Ograniczenie_I3_max"

minimalizacjaStrat += 1 <= I4, "Ograniczenie_I4_min"
minimalizacjaStrat += I4 <= 3, "Ograniczenie_I4_max"

minimalizacjaStrat += 3 <= I5, "Ograniczenie_I5_min"
minimalizacjaStrat += I5 <= 5, "Ograniczenie_I5_max"

# rozwiązanie problemu
minimalizacjaStrat.solve()

# status problemu
print ("Status:", LpStatus[minimalizacjaStrat.status])

for v in minimalizacjaStrat.variables():
    print (v.name, "=", v.varValue)

# wyswietlenie optymalnych wartosci rezystorow
print("R1:", 6000/I1.varValue,"Ohm")
print("R2:", 10000/I2.varValue,"Ohm")
print("R3:", 4000/I3.varValue,"Ohm")
print("R4:", 7000/I4.varValue,"Ohm")
print("R5:", 3000/I5.varValue,"Ohm")

Status: Optimal
Prad_1 = 3.0
Prad_2 = 1.0
Prad_3 = 1.0
Prad_4 = 1.0
Prad_5 = 3.0
R1: 2000.0 Ohm
R2: 10000.0 Ohm
R3: 4000.0 Ohm
R4: 7000.0 Ohm
R5: 1000.0 Ohm


## Jaki maksymalny prad może popłynąć przez układ rezystorów?

Plik: Rezystory2.png

Podane są maksymalne prądy, jakie mogą przepływać przez każdy z elementów. Należy określić jaki maksymalny prąd może przepłynąć przez cały układ.

Największy prąd jaki będzie przepływać przez układ, to prąd rezystora R3. Zatem współczynnikiem do zoptymalizowania może być prąd I3.

Muszą zostać spełnione prawa Kirchhoffa (prądowe i napięciowe).  

![](Rezystory2.png)

In [6]:
from pulp import *

#problem statement
maksPrad = LpProblem("maksPrad",LpMaximize)

#definicja zmiennych decyzyjnych
I1 = LpVariable("Prad1",0,None,LpContinuous)
I2 = LpVariable("Prad2",0,None,LpContinuous)
I3 = LpVariable("Prad3",0,None,LpContinuous)
I4 = LpVariable("Prad4",0,None,LpContinuous)
I5 = LpVariable("Prad5",0,None,LpContinuous)

# definicja wstakznika do minimalizacji (prad I3)
maksPrad += I3, "SumaPradow" # ze schematu wynika, że największy prąd (suma pozostałych) popłynie przez I3

# ograniczenia dot. prądowego prawa Kirchhoffa
maksPrad += I1 + I2 == I3, "SumaPradow12"
maksPrad += I5 + I4 == I3, "SumaPradow45"

# ograniczenia dot. napieciowego prawa kirchhoffa
maksPrad += I1*8 == I2*6,   "PrawoKirchhoffaLewegoOczko"
maksPrad += I5*8 == I4*10,  "PrawoKirchhoffaPraweOczko"

# ograniczenia dot wartosci maksymalnych pradow
maksPrad += I1 <= 2, "Ograniczenie_I1"
maksPrad += I2 <= 3, "Ograniczenie_I2"
maksPrad += I3 <= 4, "Ograniczenie_I3"
maksPrad += I4 <= 2, "Ograniczenie_I4"
maksPrad += I5 <= 2, "Ograniczenie_I5"

# rozwiazanie problemu
maksPrad.solve()
print ("Status:", LpStatus[maksPrad.status])

# wyswietlenie wynikow
for v in maksPrad.variables():
    print (v.name, "=", v.varValue)

print("Maksymalny prad plynacy przez uklad wynosi",I3.varValue,"A")

Status: Optimal
Prad1 = 1.5428571
Prad2 = 2.0571429
Prad3 = 3.6
Prad4 = 1.6
Prad5 = 2.0
Maksymalny prad plynacy przez uklad wynosi 3.6 A
